# Свертка и предвыборка

1. Построить нейросети с линейными, сверточными слоями и слоями подвыборки.
2. Проверить, какая конфигурация работает лучше линейных слоев.
3. Оценить качество предсказания по коэффициенту сходства.

## Подключение библиотек.

In [2]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras

from sklearn.model_selection import train_test_split
from skimage import io
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D